# Week 10 - Class 9

This notebook have the most important exercises for theoretical class 9, which covers chapter 8, and 9 from the book. The labs from https://github.com/fchollet/deep-learning-with-python-notebooks are a guide, mainly the labs 8,9. The topic in scope from Chapter 7 topics are also included in this lab.

It will be covered:
- Read image data (separate into folders, resize, normalize)
- Image Classification
- Data Augmentation


In [1]:
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array

from os import listdir
from PIL import Image
import numpy as np
from os.path import isfile, join
import os, shutil, pathlib

# Image Classification

Image dataset from here: https://www.kaggle.com/competitions/dogs-vs-cats/overview

## Data processing

In [2]:
#UPLOAD kaggle.json (IN GITHUB)
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats
!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test1.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace train/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace train/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [3]:
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        try:
          os.makedirs(dir)
        except:
          print(dir, 'created')
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

cats_vs_dogs_small/train/cat created
cats_vs_dogs_small/train/dog created
cats_vs_dogs_small/validation/cat created
cats_vs_dogs_small/validation/dog created
cats_vs_dogs_small/test/cat created
cats_vs_dogs_small/test/dog created


In [4]:
# Find the average image size
# all NN receive a fixed size (the real datasets have images with multiple sizes)
path_cat=pathlib.Path('cats_vs_dogs_small/train/cat')
path_dog=pathlib.Path('cats_vs_dogs_small/train/dog')

onlyfiles_cat = [f for f in listdir(path_cat) if isfile(join(path_cat, f))]
onlyfiles_dog = [f for f in listdir(path_dog) if isfile(join(path_dog, f))]

In [5]:
cats_size=[Image.open(join(path_cat, filename)).size for filename in onlyfiles_cat]
dogs_size=[Image.open(join(path_dog, filename)).size for filename in onlyfiles_dog]

In [6]:
# PRINT THE MEAN, MIN, MAX, MEDIAN FOR THE IMAGES OF CATS AND DOGS

cats_width=<your-answer>
print(np.mean(cats_width), np.min(cats_width), np.max(cats_width), np.median(cats_width))

SyntaxError: ignored

In [ ]:
#sol
cats_width=[k[0] for k in cats_size]
print(np.mean(cats_width), np.min(cats_width), np.max(cats_width), np.median(cats_width))

In [ ]:
#sol
cats_height=[k[1] for k in cats_size]
print(np.mean(cats_height), np.min(cats_height), np.max(cats_height), np.median(cats_height))

In [ ]:
#sol
dogs_width=[k[0] for k in dogs_size]
print(np.mean(dogs_width), np.min(dogs_width), np.max(dogs_width), np.median(dogs_width))

In [ ]:
#sol
dogs_height=[k[1] for k in dogs_size]
print(np.mean(dogs_height), np.min(dogs_height), np.max(dogs_height), np.median(dogs_height))

In [ ]:
# How to do it with image_dataset_from_directory
new_base_dir = pathlib.Path("cats_vs_dogs_small")

from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)



In [ ]:
# BUILD A DATASET
import numpy as np
import tensorflow as tf
random_numbers = np.random.normal(size=(1000, 16))
dataset = tf.data.Dataset.from_tensor_slices(random_numbers)

In [ ]:
#ACCESS DATASET SHAPE AND ELEMENTS
for i, element in enumerate(dataset):
    print(element.shape)
    if i >= 2:
        break

In [ ]:
# SEE THE SHAPE OF TRAIN DATASET ELEMENTS


In [ ]:
#sol
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

The Keras **callbacks** API will help you transform your call to model.fit() 

A callback is an object (a class instance implementing specific methods) that is passed to the model in the call to **fit()** and that is called by the model at various points
during training. **It has access to all the available data about the state of the model and its performance, and it can take action: interrupt training, save a model, load a different weight set, or otherwise alter the state of the model.**

Here are some examples of ways you can use callbacks:
- *Model checkpointing* — Saving the current state of the model at different points during training.
- *Early stopping* — Interrupting training when the validation loss is no longer improving (and of course, saving the best model obtained during training).
- *Dynamically adjusting the value of certain parameters during training* — Such as the learning rate of the optimizer.
- *Logging training and validation metrics during training, or visualizing the representations learned by the model as they’re updated* — The fit() progress bar that you’re
familiar with is in fact a callback!

The keras.callbacks module includes a number of built- in callbacks (this is not an exhaustive list):
- keras.callbacks.ModelCheckpoint
- keras.callbacks.EarlyStopping
- keras.callbacks.LearningRateScheduler
- keras.callbacks.ReduceLROnPlateau
- keras.callbacks.CSVLogger

In [ ]:
#MNIST Example

from tensorflow.keras.datasets import mnist

#DEFINE THE MODEL: HEFE WE HAVE A FUNCTIONAL API MODEL
def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model
#DATA PROCESSING: RESHAPE, NORMALIZE, CREATE DATASETS
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

model = get_mnist_model()
#DEFINE METRICS
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
#TRAIN
model.fit(train_images, train_labels,
          epochs=3,
          validation_data=(val_images, val_labels))
#EVALUATE
test_metrics = model.evaluate(test_images, test_labels)
#PREDICT
predictions = model.predict(test_images)

In [ ]:
# USING CALLBACKS
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=2,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True,
    )
]
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels,
          epochs=10,
          callbacks=callbacks_list,
          validation_data=(val_images, val_labels))

In [ ]:
#BUILD A MODEL FOR THE CATS AND DOGS DATASET WITH:
#HEIGHT AND WIDTH IS 180 PIXELS
#ALL KERNELS HAVE A SIZE OF 3
#ALL ACTIVATION FUNCTIONS ARE "relu" except the last one which is "sigmoid"

#input layer, check the input size

#normalization layer (tip: how do you normalize rgb images?)

#block A
# 2D convolution layer with 32 filters

#maxpooling layer with a size of 2

#block B
# 2D convolution layer with 64 filters

#maxpooling layer with a size of 2

#block C
# 2D convolution layer with 128 filters

#maxpooling layer with a size of 2

#block D
# 2D convolution layer with 256 filters

#maxpooling layer with a size of 2

#block D
# 2D convolution layer with 256 filters

#data transformation layer

#fully connected layer with X outputs (X is the number of classes the CNN is predicting)

#attach previous sequence of layers to keras.Model class

# compile the model with the optimizer='rmsprop'; choose the loss and metrics accordingly 



In [ ]:
#sol
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
#CALLBACKS

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss"),

        keras.callbacks.TensorBoard(
    log_dir="/LOG",
)
]

#data aug
#pre trained
# number of parameters (slide from classes)
#x ception example
#put link for visualizations


In [ ]:
history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=validation_dataset,
    callbacks=[callbacks])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /LOG

In [ ]:
test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "ticket_classifier_with_shape_info.png", show_shapes=True)



In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

## Data Augmentation

1.7 Why are we doing data augmentation? Connect your answer with the validation and training loss trends through epochs.

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[2].numpy().astype("uint8"))
        plt.axis("off")

## Regularization


Defining a new convnet that includes image augmentation and dropout



In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
#with 15min of training (100 epochs), data augmentation and dropout 
# went from 70% to 77% in val acc (without adding data!)
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

# Transfer Learning with data augmentation

In [ ]:
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
conv_base.trainable = False

In [ ]:
conv_base.trainable = True
print("This is the number of trainable weights "
      "before freezing the conv base:", len(conv_base.trainable_weights))

In [ ]:
conv_base.trainable = False
print("This is the number of trainable weights "
      "after freezing the conv base:", len(conv_base.trainable_weights))

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x) #the data passes 1 time through the weight
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
test_model = keras.models.load_model(
    "feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

# Fine Tuning

2.0 What is the difference between transfer learning and fine tuning?

In [ ]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
model = keras.models.load_model("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")